In [9]:
include("data/10_ulysses_3.tsp")
n,L,B,K,W_v,w_v,W,coordinates

(10, 2, 87, 3, [1.47239, 0.82384, 0.147042, 1.11452, 1.35826, 1.95929, 1.98969, 1.54618, 2.14362, 1.6074], [4, 14, 3, 2, 20, 5, 14, 10, 10, 18], 9, [38.24 20.42; 39.57 26.15; … ; 41.23 9.1; 41.17 13.05])

In [15]:
using JuMP
using CPLEX

In [8]:
function distance(n::Int64,coordinates::Matrix{Float64})
    l=Matrix{Float64}(zeros(n,n))
    for i in 1:n
        for j in 1:n
            l[i,j]=sqrt((coordinates[i,1]-coordinates[j,1])^2+(coordinates[i,2]-coordinates[j,2])^2)
        end 
    end 
    return l
end

distance (generic function with 1 method)

In [ ]:
l=distance(n,coordinates)

In [23]:
function resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
    
    l=distance(n,coordinates)
    m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2))
    @variable(m,x[i=1:n,j=1:n],Bin)
    @variable(m,y[i=1:n,k=1:K],Bin)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[k=1:K],sum(w_v[i]*y[i,k] for i=1:n)<=B)
    @constraint(m,[i=1:n],sum(y[i,k] for k=1:K)==1)
    @objective(m,Min,sum(l[i,j]*x[i,j] for i=1:n,j=1:n))
    return m 
end

resolution_statique (generic function with 2 methods)

In [24]:
function resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
    l=distance(n,coordinates)
    m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2))
    @variable(m,alpha>=0)
    @variable(m,beta[i=1:n,j=1:n]>=0)
    @variable(m,x[i=1:n,j=1:n],Bin)
    @variable(m,y[i=1:n,k=1:K],Bin)
    @variable(m,gamma[k=1:K]>=0)
    @variable(m,phi[i=1:n,k=1:K]>=0)



    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
    @constraint(m,[i=1:n,j=1:n],alpha+beta[i,j]>=(lh[i]+lh[j])*x[i,j])
    @constraint(m,[k=1:K],sum(w_v[i]*y[i,k]+W_v[i]*phi[i,k] for i=1:n)+W*gamma[k]<=B)
    @constraint(m,[i=1:n,k=1:K],gamma[k]+phi[i,k]>=w_v[i]*y[i,k])
    @constraint(m,[i=1:n],sum(y[i,k] for k=1:K)==1)

    @objective(m,Min,sum(l[i,j]*x[i,j]+3*beta[i,j] for i=1:n,j=1:n)+L*alpha)
    return m
end

resolution_dualite (generic function with 2 methods)

In [25]:
include("data/10_ulysses_3.tsp")
ms=resolution_statique(n,L,B,K,W_v,w_v,W,coordinates)
optimize!(ms)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
Infeasibility row 'c1':  0  = 1.
Presolve time = 0.00 sec. (0.16 ticks)


In [22]:
l=distance(n,coordinates)
m=Model(optimizer_with_attributes(CPLEX.Optimizer, "CPX_PARAM_MIPDISPLAY" =>2))
@variable(m,x[i=1:n,j=1:n],Bin)
@variable(m,y[i=1:n,k=1:K],Bin)
@constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]+y[i,k]+y[j,k]<=1)
@constraint(m,[i=1:n,j=1:n,k=1:K],x[i,j]-y[i,k]+y[j,k]<=1)
@constraint(m,[i=1:n,j=1:n,k=1:K],-x[i,j]+y[i,k]+y[j,k]<=1)
@constraint(m,[k=1:K],sum(w_v[i]*y[i,k] for i=1:n)<=B)
@constraint(m,[i=1:n],sum(y[i,k] for k=1:n)==1)
@objective(m,Min,sum(l[i,j]*x[i,j] for i=1:n,j=1:n))
optimize!(m)

BoundsError: BoundsError: attempt to access 10×3 Matrix{VariableRef} at index [1, 4]

In [26]:
md=resolution_dualite(n,L,B,K,W_v,w_v,W,coordinates)
optimize!(md)


Root node processing (before b&c):
  Real time             =    0.00 sec. (0.24 ticks)
Parallel b&c, 8 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.24 ticks)
Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
Infeasibility row 'c1':  0  = 1.
Presolve time = 0.00 sec. (0.19 ticks)
